In [1]:
import pymongo
from TT import MACD,RSI,KDJ
import pandas as pd

In [ ]:
# 数据库相关

import pymongo

DCE_DB_NAME = "DCEdb"
DCE_COMMODITY_PRICE_COLLECTION_NAME = "CommodityPriceDb"
DCE_FUTURE_PRICE_COLLECTION_NAME = "DCEFuturePriceDb"
DCE_FUTURE_CODE_COLLECTION_NAME = 'DCEFutureCodeDb'
COSMOS_CONNECTION_STRING = 'mongodb://lei:WMNi1Hh4vyOVG4V3OLcyaoZwOOl7YVHUUiKFeuGKbiS9rytxi2qd2VpnekCYj8IVrN6AFVHvLBUxACDb7vuFnA==@lei.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@lei@'

client = pymongo.MongoClient(COSMOS_CONNECTION_STRING)
DCEdb = client[DCE_DB_NAME]

DCE_future_price_collection = DCEdb[DCE_FUTURE_PRICE_COLLECTION_NAME]
DCE_commodity_price_collection = DCEdb[DCE_COMMODITY_PRICE_COLLECTION_NAME]


In [ ]:
def get_future_daily_price(start=None, end=None,code=[]):
    '''查询commodityprice表，获取某个商品或代码的历史现价，基差等
    '''
    if len(code)!=0:
        future_price_docs = DCE_commodity_price_collection.find(
                            {"$and":[
                                {'date':{'$gte':start}},
                                {'date':{'$lte':end}},
                                {'main_future_code':{'$in':code}}]}
                            ).sort("update_time", pymongo.ASCENDING)
    else:
        future_price_docs = DCE_commodity_price_collection.find(
                            {"$and":[
                                {'date':{'$gte':start}},
                                {'date':{'$lte':end}}]}
                            ).sort("update_time", pymongo.ASCENDING)
    return future_price_docs

In [ ]:
def get_future_price(start=None,end=None,code=[],):
    '''获取某个现货的相关信息
    '''
    if len(code)!=0:
        future_price_docs = DCE_future_price_collection.find(
                            {"$and":[
                                {'date':{'$gte':start}},
                                {'date':{'$lte':end}},
                                {'future_code':{'$in':code}}]}
                            ).sort("update_time", pymongo.ASCENDING)
    else:
        future_price_docs = DCE_future_price_collection.find(
                            {"$and":[
                                {'date':{'$gte':start}},
                                {'date':{'$lte':end}}]}
                            ).sort("update_time", pymongo.ASCENDING)
    return future_price_docs

In [ ]:
# 检查数据库查询结果
info = pd.DataFrame([i for i in get_future_daily_price('2023-09-08','2023-09-10',['CU2003'])])
print(info)

In [ ]:
# info = pd.DataFrame([i for i in get_future_price('2023-09-12','2023-09-12',['J2401']) if 90000<= i['clock'] <=110000])
info = pd.DataFrame([i for i in get_future_daily_price('2001-01-01','2022-01-01',['CU2003'])])
info.head() 

In [2]:
# 获取某个期货的历史每日信息，KDJ,RSI,MACD的计算都需要每日的数据，下面是临时获取方法：
start = '1978-01-01'
end = '2025-01-01'
# 需要其他期货数据，修改此处代码
code = 'J2310'

allpage = 300
# page_url = 'https://vip.stock.finance.sina.com.cn/q/view/vFutures_History.php?page='+str(2)+'&breed='+code+'&type=inner&start='+start+'&end='+end

In [3]:
# 下面为KDJ等参数的计算，由于数据库中暂时不包含一个期货所有日期的数据，因此临时爬取这部分数据，数据爬取后放到blanklist中

In [4]:
import requests
import os
import datetime
import pandas as pd
from bs4 import BeautifulSoup
blanklist = []
from tqdm import trange
for p in trange(allpage):
    # print(f'page:{p}')
    page_url = f'https://vip.stock.finance.sina.com.cn/q/view/vFutures_History.php?page='+str(p)+'&breed='+code+'&type=inner&start='+start+'&end='+end
    # print(page_url)
    r = requests.get(page_url)
    soup = BeautifulSoup(r.text.encode('utf8'), 'html.parser')
    table = soup.find_all("div", class_="historyList")
    table = BeautifulSoup(table[0].prettify().encode('utf8'), 'html.parser')
    a = pd.read_html(table.prettify())


    for index,row in a[0].iterrows():
        if index>1:
            blanklist.append([row[0],float(row[1]),float(row[2]),float(row[3]),float(row[4]),int(row[5])])

100%|██████████| 300/300 [01:08<00:00,  4.40it/s]


In [19]:
# 爬取的数据一共6列，日期，收盘，开盘，最高，最低，成交量
df = pd.DataFrame(blanklist, columns =['date', 'close','open','high','low','deal'])
df = df[df['close']>0.1]

In [6]:
import plotly.express as px

In [7]:
# KDJ
# KDJ指标是一种基于随机指标（K值）、平滑指标（D值）和J值（相对于D值的3倍数）计算而来的技术指标。
# KDJ指标通过测量收盘价与最高价和最低价之间的关系，来衡量价格的强势或弱势，并判断市场的超买超卖情况。
# K = (C - L) / (H - L) × 100
# D = MA(K, M)
# J = 3 * D - 2 * K
# 具体查看TT.py计算
# params:
# close,high,low,N,M1,M2

In [20]:
k,d,j = KDJ(df.close,df.high,df.low)

In [21]:
# MACD
# DIF：简称快线，是快速移动平均线，即正负差，DIF值=快速平滑移动平均线（EMA1）— 慢速平滑移动平均线（EMA2）
# DEA：简称慢线，是慢速移动平均线，也叫异同平均数，算法就是无数个DIF值的加权平均数
# 

In [22]:
dif,dea,macd = MACD(df.close)

In [23]:
# RSI

In [24]:
rsi = RSI(df.close,24)

In [33]:
l = [df.date,k,d,j,dif,dea,macd,rsi]
indicators = pd.DataFrame(list(map(list, zip(*l))),columns =['date','k', 'd','j','dif','dea','macd','rsi'])

In [41]:
# df = px.data.gapminder().query("continent=='Oceania'")

fig = px.line(indicators, x="date", y="macd",  title="low price")
fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()